# 실습 - Non-Personalized Recommendation - Best Seller 추천

- 영화 데이터(movies.csv) 파일을 Best Seller 추천 방식에 적용합니다.  
- 모든 사용자에게 가장 인기 있는 도일한 영화를 추천합니다.

In [1]:
# 필요한 라이브러리 임포트
import pandas as pd  
import numpy as np  \

# from sklearn.metrics import root_mean_squared_error   # sklearn 최신 버전
from sklearn.metrics import mean_squared_error  # sklearn 구 버전

# 경고 메시지를 무시하도록 설정
import warnings
warnings.filterwarnings('ignore')

# sklearn 라이브러리의 버전 확인
import sklearn
sklearn.__version__

'1.2.2'

## 영화 데이터(movies.csv) 파일을 DataFrame으로 읽기

In [2]:
# movies.csv 파일 읽기
movies = pd.read_csv('data/ml-latest-small/movies.csv', index_col=0)
movies.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


## 사용자 평점 (ratings.csv) 파일을 DataFrame으로 읽기 

In [3]:
# ratings.csv 파일 읽기
ratings = pd.read_csv('data/ml-latest-small/ratings.csv', index_col=0)
ratings.head()

,movieId,rating,timestamp
userId,,,
1,1,4.0,964982703
1,3,4.0,964981247
1,6,4.0,964982224
1,47,5.0,964983815
1,50,5.0,964982931


# 베스트셀러 추천 방식

- 이 방식은 모든 사용자에게 가장 인기 있는 동일한 영화를 추천합니다.
- 상품에 대한 모든 평가의 평균을 계산하고, 이 평균 점수가 높은 순서대로 상품을 추천합니다.

In [4]:
# 각 영화별 평균 평점 계산
movie_mean_rating = ratings.groupby('movieId')['rating'].mean()
movie_mean_rating.head()

movieId
1    3.920930
2    3.431818
3    3.259615
4    2.357143
5    3.071429
Name: rating, dtype: float64

In [5]:
# 평균 평점이 가장 높은 상위 5개 영화 찾기
top_movies = movie_mean_rating.sort_values(ascending=False).head(5)
top_movies.head()

movieId
88448     5.0
100556    5.0
143031    5.0
143511    5.0
143559    5.0
Name: rating, dtype: float64

In [6]:
# 상위 5개 영화의 제목 찾기
top_movies_titles = movies.loc[top_movies.index, 'title']
top_movies_titles

movieId
88448     Paper Birds (Pájaros de papel) (2010)
100556               Act of Killing, The (2012)
143031                          Jump In! (2007)
143511                             Human (2015)
143559                      L.A. Slasher (2015)
Name: title, dtype: object

## 추천 시스템의 정확도 측정

In [13]:
rmse = []

for user in set(ratings.index): # 모든 userId 에 대해 순회
    y_true = ratings.loc[user, 'rating']  # 현재 사용자가 평가한 영화 평점
    y_pred = movie_mean_rating.loc[ratings.loc[user, 'movieId']]  # 해당 영화에 대한 모든 사용자의 평균 평점
    assert len(y_true) == len(y_pred), "length different"  # 동일한 길이인지 check
    
    #loss = root_mean_squared_error(y_true, y_pred)    # sklearn latest version
    loss = mean_squared_error(y_true, y_pred, squared=True)    # sklearn old version
    
    rmse.append(loss)

print("Best Seller 추천 방식의 RMSE = ", np.mean(rmse))

Best Seller 추천 방식의 RMSE =  0.9008684066524535
